In [1]:
import numpy as np
import random
from tabulate import tabulate
from tqdm import tqdm
import ast
import autograd.numpy as np
from autograd import grad

In [2]:
def generate_grid_world(length, width,path_lenght,holes_number,Random_State):
    
    random.seed(Random_State)
    #store all cells in a list
    Grid_Cells = []
    for row in range(length):
        for col in range(width):
            Grid_Cells.append([row,col])


    #specify the number of holes in the gridworld
    
    #specify the start point as a random cell
    start = [random.randint(0, length), random.randint(0, width)]

    #create a path from start point
    """instead of defining start and goal points,
      we define just a start point and a random path with a random lenght to
       another point and name it as goal point"""
    
    def random_path(Start, Path_Lenght,length, width):
        
        Path = []
        Path.append(Start)
        for i in range(Path_Lenght):
            
            #there are two moves that take us on a random cell named Goal [1,0], [0,1]
            
            move = random.choice([[1,0], [0,1]])
            
            #update the start cell/point by the above move
            Start = [x + y for x, y in zip(Start, move)]
            
            #if the movement take us out of our gridworld, we reverse the change in the start point
            if Start[0] < 0 or Start[1] < 0 or Start[0] > length-1 or Start[1] > width-1:

                Start = [x - y for x, y in zip(Start, move)]

            else:
                
                #create a path history
                Path.append(Start)

        Goal = Start

        return Goal,Path
    

    GoalPath = random_path(start, path_lenght,length, width)

    goal = GoalPath[0]
    path = GoalPath[1]

    #now we must eliminate the path cells from the Grid_Cells to choose hole cells from remaining cells

    FreeCells = [x for x in Grid_Cells if x not in path]

    Holes = random.sample(FreeCells, holes_number)

    #Also, we can visualize our gridworld in a simple way

    def mark_holes(holes):
        marked_data = [["Hole" if [row, col] in holes else [row, col] for col in range(width)] for row in range(length)]
        return marked_data
    
    marked_matrix = mark_holes(Holes)

    print(tabulate(marked_matrix, tablefmt="grid"))

    
    return length, width, start, goal, Holes, path,Grid_Cells

In [3]:
#environment = generate_grid_world(50, 40,1300,400,39)
environment = generate_grid_world(5, 4,4,4,39)

environment

+--------+--------+--------+--------+
| Hole   | [0, 1] | [0, 2] | [0, 3] |
+--------+--------+--------+--------+
| [1, 0] | [1, 1] | [1, 2] | [1, 3] |
+--------+--------+--------+--------+
| Hole   | [2, 1] | [2, 2] | [2, 3] |
+--------+--------+--------+--------+
| Hole   | [3, 1] | Hole   | [3, 3] |
+--------+--------+--------+--------+
| [4, 0] | [4, 1] | [4, 2] | [4, 3] |
+--------+--------+--------+--------+


(5,
 4,
 [1, 2],
 [4, 3],
 [[2, 0], [3, 2], [3, 0], [0, 0]],
 [[1, 2], [1, 3], [2, 3], [3, 3], [4, 3]],
 [[0, 0],
  [0, 1],
  [0, 2],
  [0, 3],
  [1, 0],
  [1, 1],
  [1, 2],
  [1, 3],
  [2, 0],
  [2, 1],
  [2, 2],
  [2, 3],
  [3, 0],
  [3, 1],
  [3, 2],
  [3, 3],
  [4, 0],
  [4, 1],
  [4, 2],
  [4, 3]])

In [4]:
def probability_distribution(grid_size,randomness):
    #random.seed(40)
    
    #by this function we generate probabilities which their sum is equal to 1
    def generate_probabilities(n):

        numbers = [random.random() for _ in range(n)]
        total_sum = sum(numbers)
        scaled_numbers = [num / total_sum for num in numbers]
        
        return scaled_numbers
    
    cells_prob = {}
    if randomness == 'stochastic':
        for cell in range(grid_size):
            
            #we set the number of probs to 4 due to 4 possible action for each cell (go to its neighbors)
            probs = generate_probabilities(4)

            cells_prob[cell] = probs
    elif randomness == 'equal probable':

        for cell in range(grid_size):

            cells_prob[cell] = [0.25,0.25,0.25,0.25]
    
    elif randomness == 'deterministic':
        for cell in range(grid_size):

            cells_prob[cell] = [0.03,0.06,0.01,0.9] #[0,0,0,1] ##[0.15,.15,0.1,0.6]


    #Note that we consider the correspondence between probabilities and actions as below:
    #probs = [p1, p2, p3, p4] ---> [[1,0],[-1,0],[0,1],[0,-1]]

    return cells_prob

def neighbor_cells(cell):

    grid_cells = environment[6]
    Actions = [[1,0],[-1,0],[0,1],[0,-1]]

    Neighbors = []
    Actions_Neighbors = []
    for action in Actions:

        neighbor = [x + y for x, y in zip(cell, action)]
        #if neighbor not in environment[4]:
        Neighbors.append(neighbor)
        Actions_Neighbors.append(action)

    return Neighbors, Actions_Neighbors

def arbitrary_policy(randomness):

        #random.seed(randomness)
        
    policy = {}
    policy_action = {}
    for state in environment[6]:

        if state not in environment[4]:

            neighbors = neighbor_cells(state)[0]
            Actions_Neighbors = neighbor_cells(state)[1]

            allowed_positions = []

            for neighbor in neighbors:
                
                if neighbor in environment[6] and neighbor not in environment[4]:
                    
                    allowed_positions.append(neighbor)
            
            if len(allowed_positions) > 0:
                
                next_state = random.choice(allowed_positions)
                row = next_state[0] - state[0]
                col = next_state[1] - state[1]
                PolicyAction = [row, col]

                policy['{}'.format(state)] = next_state
                policy_action['{}'.format(state)] = PolicyAction



    return policy, policy_action

def state_reward(next_state):

    if next_state in environment[4]:

        r = -3
    
    elif next_state == environment[3]:

        r = 10
    
    elif next_state not in environment[6]:

        r = -2
    
    else:

        r = -1
    
    return r

def reverse_dictionary(dict):
    reverse_dict = {}
    for key in list(dict.keys()):
        val = dict[key]
        reverse_dict[val] = key
    return reverse_dict


state_indice_dict = {}
counter = 0
for state in environment[6]:

    state = str(state)
    state_indice_dict[state] = counter
    counter = counter + 1

def generate_trajectory(policy,randomness,environment_stochasticity):

    policy_action = policy[1]
    probs = probability_distribution(environment[0]*environment[1],environment_stochasticity)
    start = environment[2]
    terminate = start
    trajectory = []
    pure_trajectory = [start]
    c = 0
    while terminate != environment[3]:
        random.seed(randomness+c)
        Actions = [[1,0],[-1,0],[0,1],[0,-1]]
        action = policy_action[str(terminate)]
        Actions.remove(action)
        sorted_actions = Actions + [action]
        state_indice = state_indice_dict[str(terminate)]
        actions_prob = probs[state_indice]
        actions_prob.sort()

        selected_action = random.choices(sorted_actions, actions_prob)[0]
        current_state = terminate
        next_state = [x + y for x, y in zip(terminate, selected_action)]
        pure_trajectory.append(next_state)
        
        #if the agent goes out of the gridworld, it stays in its current state
        if next_state not in environment[6]:
            next_state = terminate
        
        #if it drops into the holes, it goes to the start points
        elif next_state in environment[4]:
            next_state = start  

        terminate = next_state
        trajectory.append((current_state))
        c = c+1
    
    trajectory.append((environment[3]))
    pure_trajectory.append(environment[3])

    return trajectory,pure_trajectory

## Gradient Monte Carlo Algorithm for Estimating $\hat{v} \approx v_{\pi}$

As we want to use stochastic gradient-descent for estimating the value function, we need to define the $\hat{v}$ differentiable and also we need to extract features from each states which are used as $w_t$.

In [5]:
def extract_features(state):

    goal = environment[3]
    max_length = environment[0]
    max_width = environment[1]

    w1 = (goal[0] - state[0]) / max_width
    w2 = (goal[1] - state[1]) / max_length

    return abs(w1), abs(w2)


extract_features([2,10])

def differentiable_function(state):

    w1, w2 = extract_features(state)

    """
    Here we want to dedicate more values to the states that are closer to the terminal/goal state.
    Therefore we use Cos() function that when the distance between state and the terminal is short,
    the function return a higher value. Another advantage of this function is that it change between 
    -1 and 1 and here in [0,1). Also consider that this function is differentiable.
    """
    return np.cos(w1 + w2)

In [17]:
def gradient_monte_carlo(num_trials, policy, gamma, alpha, environment_stochasticity):

    W = {}
    for state in environment[6]:

        if state not in environment[4]:

            Features = extract_features(state)
            
            W[str(state)] = [Features[0]+random.uniform(1e-9, 1e-8), Features[1]+random.uniform(1e-9, 1e-8)]
    #print(W)
            #W[str(state)] = [random.uniform(1e-9, 1e-8), random.uniform(1e-9, 1e-8)]

    V = {}
    state_observed = {}
    for state in environment[6]:

        if state not in environment[4]:
            
            V[str(state)] = 0
            state_observed[str(state)] = 0
    
    
    for trial in tqdm(range(num_trials)):

        TRAJECTORY = generate_trajectory(policy,trial,environment_stochasticity)
        

        trajectory = TRAJECTORY[0]

        #print(trajectory)

        #total reward
        G = 0

        trajectory.reverse()
        
        
        returns = {}

        for state in environment[6]:
            
            if state not in environment[4]:# and state != environment[3]:

                returns[str(state)] = 0

        first_visit = []
        G_store = {}
        for step_indx in range(len(trajectory[1:])):

            step = trajectory[1:][step_indx]
            next_step = trajectory[step_indx]

            #if step not in first_visit:

            #    first_visit.append(step)

            r = state_reward(next_step)

            G = gamma * G + r

            G_store[len(trajectory[1:])-1 - step_indx] = G
            #print(step,next_step,r)

            #returns[str(step)] = returns[str(step)] + G
        
        for step_indx in range(len(trajectory[1:])):

            step = trajectory[1:][step_indx]

            gradient_w1 = -np.sin(abs(W[str(step)][0]) + abs(W[str(step)][1])) * (W[str(step)][0]/abs(W[str(step)][0]))
            gradient_w2 = -np.sin(abs(W[str(step)][1]) + abs(W[str(step)][0])) * (W[str(step)][1]/abs(W[str(step)][1]))

            W[str(step)][0] = W[str(step)][0] +\
                    alpha * (G_store[step_indx] - np.cos(abs(W[str(step)][0]) + abs(W[str(step)][1]))) * gradient_w1

            W[str(step)][1] = W[str(step)][1] +\
                    alpha * (G_store[step_indx] - np.cos(abs(W[str(step)][0]) + abs(W[str(step)][1]))) * gradient_w2

            
            V[str(step)] = V[str(step)] + np.cos(abs(W[str(step)][0]) + abs(W[str(step)][1]))

            state_observed[str(step)] = state_observed[str(step)] + 1
    
    for state in environment[6]:

        if state not in environment[4]:

            if state_observed[str(state)] > 0:

                V[str(state)] = V[str(state)] / state_observed[str(state)]
        
    return V

In [7]:
policy_0 = arbitrary_policy(41)
gradient_monte_carlo(10000, policy_0, 0.9, 0.3, 'deterministic')

{'[0, 1]': [1.000000005620296, 0.40000000369817756], '[0, 2]': [1.0000000074459667, 0.2000000051415558], '[0, 3]': [1.000000003317392, 3.8858190930486515e-09], '[1, 0]': [0.7500000010047404, 0.6000000073802285], '[1, 1]': [0.7500000014094153, 0.4000000026922277], '[1, 2]': [0.7500000065771341, 0.20000000657948785], '[1, 3]': [0.7500000065690965, 6.702079068337278e-09], '[2, 1]': [0.5000000082078806, 0.4000000095025256], '[2, 2]': [0.5000000097102582, 0.20000000448713215], '[2, 3]': [0.5000000041594808, 2.2227509973706322e-09], '[3, 1]': [0.25000000488517904, 0.40000000255240314], '[3, 3]': [0.2500000094044007, 7.781837572133848e-09], '[4, 0]': [3.1196463043520186e-09, 0.6000000072253678], '[4, 1]': [7.2853368489074725e-09, 0.4000000018924485], '[4, 2]': [8.281856965237043e-09, 0.20000000759927017], '[4, 3]': [5.2527911866170825e-09, 9.494938132290488e-09]}


100%|██████████| 10000/10000 [00:27<00:00, 369.92it/s]


{'[0, 1]': 0.16063691334701274,
 '[0, 2]': 0.15244690878397646,
 '[0, 3]': 0.1672939981296013,
 '[1, 0]': 0.1294245780463014,
 '[1, 1]': 0.09816397975623777,
 '[1, 2]': 0.05599246043874623,
 '[1, 3]': 0.0016745852004683166,
 '[2, 1]': 0.15400605931048797,
 '[2, 2]': 0.130127768415234,
 '[2, 3]': 0.1561645925586206,
 '[3, 1]': 0.12311810824658126,
 '[3, 3]': 0.15916923257791726,
 '[4, 0]': -0.08786200322741104,
 '[4, 1]': 0.11278695057043417,
 '[4, 2]': 0.09539302458747274,
 '[4, 3]': 0}

## Semi-gradient TD(0) for Estimating $\hat{v} \approx v_{\pi}$

In [11]:
def semi_gradient_TD(num_trials, policy, gamma, alpha, environment_stochasticity):

    W = {}
    for state in environment[6]:

        if state not in environment[4]:

            Features = extract_features(state)
            
            W[str(state)] = [Features[0]+random.uniform(1e-9, 1e-8), Features[1]+random.uniform(1e-9, 1e-8)]
    #print(W)
            #W[str(state)] = [random.uniform(1e-9, 1e-8), random.uniform(1e-9, 1e-8)]

    V = {}
    state_observed = {}
    for state in environment[6]:

        if state not in environment[4]:
            
            V[str(state)] = 0
            state_observed[str(state)] = 0
    
    
    for trial in tqdm(range(num_trials)):

        TRAJECTORY = generate_trajectory(policy,trial,environment_stochasticity)
        

        trajectory = TRAJECTORY[0]

        for step_indx in range(len(trajectory[:-1])):

            step = trajectory[step_indx]
            next_step = trajectory[step_indx+1]

            r = state_reward(next_step)

            gradient_w1 = -np.sin(abs(W[str(step)][0]) + abs(W[str(step)][1])) * (W[str(step)][0]/abs(W[str(step)][0]))
            gradient_w2 = -np.sin(abs(W[str(step)][1]) + abs(W[str(step)][0])) * (W[str(step)][1]/abs(W[str(step)][1]))

            W[str(step)][0] = W[str(step)][0] +\
                    alpha * (r + gamma * np.cos(abs(W[str(next_step)][0]) + abs(W[str(next_step)][1]))\
                         - np.cos(abs(W[str(step)][0]) + abs(W[str(step)][1]))) * gradient_w1

            W[str(step)][1] = W[str(step)][1] +\
                    alpha * (r + gamma * np.cos(abs(W[str(next_step)][0]) + abs(W[str(next_step)][1]))\
                        - np.cos(abs(W[str(step)][0]) + abs(W[str(step)][1]))) * gradient_w2

            
            V[str(step)] = V[str(step)] + np.cos(abs(W[str(step)][0]) + abs(W[str(step)][1]))


    return V

In [12]:
semi_gradient_TD(10000, policy_0, 0.9, 0.3, 'deterministic')

100%|██████████| 10000/10000 [00:30<00:00, 327.02it/s]


{'[0, 1]': -147504.33524832348,
 '[0, 2]': -157375.88353340572,
 '[0, 3]': -15253.144578771835,
 '[1, 0]': -1185.6918712058864,
 '[1, 1]': -13832.83362986481,
 '[1, 2]': -173237.7524772516,
 '[1, 3]': -165681.08252640624,
 '[2, 1]': -12563.12460659772,
 '[2, 2]': -4780.734573837872,
 '[2, 3]': -38356.501572869,
 '[3, 1]': -11266.963831699599,
 '[3, 3]': -35139.76171535326,
 '[4, 0]': -676.6452458026064,
 '[4, 1]': -11778.335194027299,
 '[4, 2]': 1099.6642694987265,
 '[4, 3]': 0}

### The relatively bad values for state = [3,3] is because of the policy_0 where [3,3] -> [2, 3].

In [15]:
semi_gradient_TD(10000, policy_0, 0.9, 0.3, 'stochastic')

100%|██████████| 10000/10000 [01:20<00:00, 124.32it/s]


{'[0, 1]': -192464.64829735464,
 '[0, 2]': -357124.49979619135,
 '[0, 3]': -324278.64738107345,
 '[1, 0]': -70349.35898796997,
 '[1, 1]': -169908.7933384972,
 '[1, 2]': -365174.5438423179,
 '[1, 3]': -277125.60024987085,
 '[2, 1]': -89912.9279441882,
 '[2, 2]': -78841.04371387436,
 '[2, 3]': -97724.72073692146,
 '[3, 1]': -42021.83523925165,
 '[3, 3]': -51563.38497072105,
 '[4, 0]': -15718.264895522268,
 '[4, 1]': -29555.51758165824,
 '[4, 2]': -6576.163885053075,
 '[4, 3]': 0}

In [18]:
gradient_monte_carlo(10000, policy_0, 0.9, 0.3, 'stochastic')

100%|██████████| 10000/10000 [01:20<00:00, 124.29it/s]


{'[0, 1]': 0.2152915323267137,
 '[0, 2]': 0.2044418567859039,
 '[0, 3]': 0.20682538223015068,
 '[1, 0]': 0.21874847057759472,
 '[1, 1]': 0.20789614935668296,
 '[1, 2]': 0.19462008123614782,
 '[1, 3]': 0.20345409586860508,
 '[2, 1]': 0.22318775380620456,
 '[2, 2]': 0.20554541307245805,
 '[2, 3]': 0.20853002608580262,
 '[3, 1]': 0.21994554657237142,
 '[3, 3]': 0.2186466067376016,
 '[4, 0]': 0.20354032453940007,
 '[4, 1]': 0.2133682301982509,
 '[4, 2]': 0.21588156309399237,
 '[4, 3]': 0}

In [ ]:
def decay_alpha(alpha):

    